In [155]:
def processQuery(query):
    q_token = createTokenOptimized(query)
    q_token = linguisticTokenOptimized(q_token)
    posting_list = []
    
    for token in q_token:
        try:
            posting_list.append(posting[token])
        except:
            print(token)
            posting_list.append([])
            break
    result = mergePostings(posting_list)
    return result

In [156]:
def processQueryShortToLong(query):
    q_token = createTokenOptimized(query)
    q_token = linguisticTokenOptimized(q_token)
    posting_list = []
    
    for token in q_token:
        try:
            posting_list.append(posting[token])
        except:
            print(token)
            posting_list.append([])
            break
    posting_list = sorted(posting_list, key=lambda x: x[0])
    result = mergePostings(posting_list)
    return result

In [151]:
def processQuerySkip(query):
    q_token = createTokenOptimized(query)
    q_token = linguisticTokenOptimized(q_token)
    posting_list = []
    
    for token in q_token:
        try:
            posting_list.append(posting[token])
        except:
            #print(token)
            posting_list.append([])
            break
    result = skp(posting_list)
    return result

In [154]:
def processQuerySkipSort(query):
    q_token = createTokenOptimized(query)
    q_token = linguisticTokenOptimized(q_token)
    posting_list = []
    
    for token in q_token:
        try:
            posting_list.append(posting[token])
        except:
            #print(token)
            posting_list.append([])
            break
    posting_list = sorted(posting_list, key=lambda x: x[0])
    result = skp(posting_list)
    return result

In [98]:
def no_of_skip(arr_list):
    return math.ceil(math.sqrt(len(arr_list)))

def skp(postingList):
    posting1 = postingList[0][1]
    
    for i in range(1,len(postingList),1):
        posting2 = postingList[i][1]
        p = 0
        q = 0
        #print(posting1)
        pskip = no_of_skip(posting1)
        qskip = no_of_skip(posting2)
        p1_len = len(posting1)
        p2_len = len(posting2)
        merged = []
        
        while p < p1_len and q < p2_len:
            if int(posting1[p]) == int(posting2[q]):
                merged.append(posting1[p])
                p += 1
                q += 1

            elif int(posting1[p]) < int(posting2[q]):
                if (p1_len > (pskip+p)) and (posting1[p+pskip] <= posting2[q]):
                    p += pskip
                else:
                    p += 1

            else:
                if (p2_len > (qskip+q)) and posting2[q+qskip] <= posting1[p]:
                    q += qskip
                else:
                    q += 1
        posting1 = merged
    return posting1def processQuery(query):
    q_token = createTokenOptimized(query)
    q_token = linguisticTokenOptimized(q_token)
    posting_list = []
    
    for token in q_token:
        try:
            posting_list.append(posting[token])
        except:
            print(token)
            posting_list.append([])
            break
    result = mergePostings(posting_list)
    return result

def processQueryShortToLong(query):
    q_token = createTokenOptimized(query)
    q_token = linguisticTokenOptimized(q_token)
    posting_list = []
    
    for token in q_token:
        try:
            posting_list.append(posting[token])
        except:
            print(token)
            posting_list.append([])
            break
    posting_list = sorted(posting_list, key=lambda x: x[0])
    result = mergePostings(posting_list)
    return result

def processQuerySkip(query):
    q_token = createTokenOptimized(query)
    q_token = linguisticTokenOptimized(q_token)
    posting_list = []
    
    for token in q_token:
        try:
            posting_list.append(posting[token])
        except:
            #print(token)
            posting_list.append([])
            break
    result = skp(posting_list)
    return result

def processQuerySkipSort(query):
    q_token = createTokenOptimized(query)
    q_token = linguisticTokenOptimized(q_token)
    posting_list = []
    
    for token in q_token:
        try:
            posting_list.append(posting[token])
        except:
            #print(token)
            posting_list.append([])
            break
    posting_list = sorted(posting_list, key=lambda x: x[0])
    result = skp(posting_list)
    return result

In [ ]:
while True:
    query = input("Enter a query (type q to exit) : ")
    if query == "q":
        break
        
    clear_output()
    start_time_non = time.time()
    
    result = processQuery(query)
    
    print("Time taken to search with non sorted order: ", time.time()-start_time_non)
    
    start_time = time.time()
    result = processQueryShortToLong(query)
    print("Time taken to search with short to long: ", time.time()-start_time)
    
    start_time_skip = time.time()
    result = processQuerySkip(query)
    print("Time taken to search with skip pointer: ", time.time()-start_time_skip)
    
    start_time_skp = time.time()
    result = processQuerySkipSort(query)
    print("Time taken to search with skip pointer and short to long sorting", time.time()-start_time_skp)
    
    print( len(result), "documents found for ", query)
    
    #for doc in result:
     #   print(os.path.basename(docId[int(doc)]))